### Sentiment analysis on reviews data
Kei Sato

ML310B - Advanced Machine Learning

March 25, 2019


We will be using reviews data to develop a sentiment analyzer, such that given a document, the model can predict if the review is positive (sentiment = 1) or negative (sentiment = 0)


In [8]:
# Load the data...
import pandas as pd
from nltk.tokenize import word_tokenize

data = pd.read_csv('data/Reviews.csv')

print("Number of positive and negative review", '\n', data["sentiment"].value_counts())
data.head()

Number of positive and negative review 
 1    25000
0    25000
Name: sentiment, dtype: int64


,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


#### Initial Text Processing
The reviews corpus has 5000 reviews and is evenly split between positive and negative reviews, so that it contains 2500 positive and 2500 negative reviews.  Before doing any more data exploration, we process the text using standard techniques.  Much of this code was taken from the Lesson 8 HW assignment.

The first step is apply some basic text processing.  This function will transform all the letters to lowercase and replace any punctuation or symbols with spaces.  At this step we will also remove English stop words.  Because this corpus contains some <br \> HTML elements, we will strip those out from the text as well.  This function will return the words in a tokenized format such each word is an element in an array.  After cleaning the text, lemmatization is applied. 

I did apply stemming to the dataset, but that produced too many non words and so it has been omitted from the text processing steps.

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Taken Lesson 8 HW assignment
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

replace_re_by_space = re.compile('[/(){}\[\]\|@,;]')
delete_re_symbols = re.compile('[^0-9a-z #+_]')
stop_words = set(stopwords.words('english'))

def combine_tokened_words(tokened_words):
    length_of_string=len(tokened_words)
    text_new=""
    for w in tokened_words:
        if w!=tokened_words[length_of_string-1]:
             text_new=text_new+w+" " # when w is not the last word so separate by whitespace
        else:
            text_new=text_new+w
    return text_new

# converts to lowercase and removes <br />, punctuation, stop words, and numbers
def text_processing(text):
    text = text.lower()
    text = text.replace("<br />", '')
    text = re.sub(replace_re_by_space.pattern, ' ', text)
    text = re.sub(delete_re_symbols.pattern, '', text)
    token_word = word_tokenize(text)
    
    # filtered_sentence contain all words that are not in stopwords dictionary
    filtered_sentence = [w for w in token_word if not w in stop_words]
    return filtered_sentence

# Lemmatizes words
def text_lemmatization(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    text = list(map(lambda word: wordnet_lemmatizer.lemmatize(word), text))
    return text

test_data = data[:500].copy(deep=True)
test_data["review"] = test_data["review"].apply(lambda text:
                                                combine_tokened_words(
                                                text_lemmatization(
                                                text_processing(text))))

#### Data exploration
Below is some initial data exploration.  We can see that the average length of positive and negative reviews is roughtly the same.  The ten most frequently occuring words are also very similar across between the sets of positive and negative reviews.  

In [54]:
import numpy as np
from collections import Counter 
from functools import reduce

# Get average length of reviews
def get_avg_length_review(data, sentiment):
    relevant_reviews = data.loc[data["sentiment"] == sentiment]["review"]
    avg_review_length = list(map(lambda review: len(review.split()), relevant_reviews))
    return int(np.mean(avg_review_length))
print("Average word count of negative reviews:", get_avg_length_review(test_data, 0))
print("Average word count of positive reviews:", get_avg_length_review(test_data, 1))

# Get 10 most frequently occuring words
def get_top_words(data, sentiment):
    relevant_reviews = data.loc[data["sentiment"] == sentiment]["review"]
    all_relevant_reviews = reduce(lambda accum, curr: accum + curr, relevant_reviews)
    return Counter(all_relevant_reviews.split()).most_common(10)
print('\n')  
print("Top 10 most common words in negative reviews", get_top_words(test_data, 0))
print("Top 10 most common words in positive reviews", get_top_words(test_data, 1))

Average word count of negative reviews: 118
Average word count of positive reviews: 120


Top 10 most common words in negative reviews [('movie', 50179), ('film', 39126), ('one', 24305), ('like', 21763), ('even', 14791), ('good', 13874), ('character', 13859), ('bad', 13580), ('time', 13497), ('would', 13403)]
Top 10 most common words in positive reviews [('film', 44641), ('movie', 39127), ('one', 25630), ('like', 17062), ('time', 14641), ('good', 14017), ('character', 13580), ('story', 13290), ('great', 12292), ('see', 11975)]


In [ ]:
# Update to pipeline???
# get test and train data
# run through tfidif vectorizer
# go through logistic regression (???) w/ cross validation
# output accuracy
def run_model_cv(data):
    data["review"] = data["review"].apply(lambda text: combine_tokened_words(
        text_lemmatization(text_processing(text))))   
    x_train, x_test, y_train, y_test = train_test_split(
        data["review"],
        data["sentiment"],
        test_size=0.3,
        random_state=42
    )
    tfid_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, use_idf=True).fit(x_train)
    x_train = tfid_vectorizer.transform(x_train)
    x_test = tfid_vectorizer.transform(x_test)

    model = LogisticRegression(solver="sag").fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("accuracy", accuracy_score(y_test, y_pred))
    
    
# run_model(data.copy(deep=True))
